In [4]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import config;
from config import db_pw;
import psycopg2;

In [5]:
#establishing the connection
conn = psycopg2.connect(
   database="Senescence_DB", user='postgres', password= db_pw, host='127.0.0.1', port= '5432'
)

#Setting auto commit false
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Retrieving data
cursor.execute('''SELECT * from birds_data_raw''')

#Fetching 1st row from the table
bird_df = pd.DataFrame(cursor.fetchall(), columns=["Collection_Num", "Bird_ID", "Sex", "Agey", "AgeClass", "BirthFPID", "U_PlateID", "RTL", "Technician", "Terr", "FPID", "Mum", "Dad", "Mac", "Pac", "BRF", "BRM"]);
print(result_df)

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

      Collection_Num  Bird_ID  Sex       Agey AgeClass  BirthFPID U_PlateID  \
0                  1     5511    0   0.246575        J        103       9_3   
1                  2     5559    1   3.597260        A        103       9_1   
2                  3     5559    1   4.715068        A        103      9_28   
3                  4     5559    1   0.657534       SA        103       9_2   
4                  5     5527    1   0.328767      OFL        103       9_3   
...              ...      ...  ...        ...      ...        ...       ...   
2657            2658      221    1   6.958904        A          8       2_1   
2658            2659      216    1   8.947945        A          8       8_8   
2659            2660      176    1   6.772603        A          8      7_32   
2660            2661      221    1   9.090411        A          8      1_12   
2661            2662      176    1  13.873973        A          8       4_3   

       RTL  Technician   Terr  FPID   Mum   Dad    

In [6]:
passover_list = [0,1]
changeRate_df = pd.DataFrame(columns=["Bird_ID", "Age_Change", "RTL_Change"])
for i in range(len(bird_df["Bird_ID"])):
    if(bird_df.loc[i,"Bird_ID"] not in passover_list):
        k = i+1
        holder_df = pd.DataFrame({"ID": [bird_df.loc[i,"Bird_ID"]], "Age": [bird_df.loc[i, "Agey"]], "RTL": [bird_df.loc[i,"RTL"]]})
        while(bird_df.loc[k,"Bird_ID"] == bird_df.loc[i,"Bird_ID"]):
            holder_df = holder_df.append({"ID": bird_df.loc[k,"Bird_ID"], "Age": bird_df.loc[k, "Agey"], "RTL": bird_df.loc[k,"RTL"]}, ignore_index=True)
            k = k+1
        if(len(holder_df) > 1):
            max_age = 0
            min_age = 0
            final_rtl = 0
            first_rtl = 0
            for row in holder_df.iterrows():
                print(row[1])
                id = row[1].ID
                if(row[1].Age == max(holder_df["Age"])):
                    max_age = row[1].Age
                    final_rtl = row[1].RTL
                elif(row[1].Age == min(holder_df["Age"])):
                    min_age = row[1].Age
                    first_rtl = row[1].RTL
            age_change = max_age - min_age
            rtl_change = final_rtl - first_rtl
            changeRate_df = changeRate_df.append({"Bird_ID": id, "Age_Change": age_change, "RTL_Change": rtl_change}, ignore_index=True)
        else:
            changeRate_df = changeRate_df.append({"Bird_ID": holder_df.loc[0,"ID"], "Age_Change": holder_df.loc[0, "Age"], "RTL_Change": holder_df.loc[0,"RTL"]}, ignore_index=True)
        passover_list.append(bird_df.loc[i,"Bird_ID"])

ID     5559.00000
Age       3.59726
RTL       0.52000
Name: 0, dtype: float64
ID     5559.000000
Age       4.715068
RTL       1.070000
Name: 1, dtype: float64
ID     5559.000000
Age       0.657534
RTL       0.990000
Name: 2, dtype: float64
ID     5539.000000
Age       1.964384
RTL       0.360000
Name: 0, dtype: float64
ID     5539.000000
Age       0.657534
RTL       0.610000
Name: 1, dtype: float64
ID     5536.000000
Age       1.252055
RTL       0.930000
Name: 0, dtype: float64
ID     5536.000000
Age       0.328767
RTL       0.470000
Name: 1, dtype: float64
ID     5536.000000
Age       4.249315
RTL       0.960000
Name: 2, dtype: float64
ID     5536.000000
Age       3.471233
RTL       0.660000
Name: 3, dtype: float64
ID     5549.000000
Age       2.589041
RTL       0.490000
Name: 0, dtype: float64
ID     5549.000000
Age       0.657534
RTL       0.370000
Name: 1, dtype: float64
ID     5509.000000
Age       1.315068
RTL       0.840000
Name: 0, dtype: float64
ID     5509.000000
Age       0.

In [7]:
changeRate_df

,Bird_ID,Age_Change,RTL_Change
0,5511.0,0.246575,1.06
1,5559.0,4.057534,0.08
2,5527.0,0.328767,1.02
3,5539.0,1.306849,-0.25
4,5536.0,3.920548,0.49
...,...,...,...
1311,163.0,9.350685,1.21
1312,494.0,4.915068,0.70
1313,4847.0,2.246575,1.01
1314,220.0,9.616438,0.69
